In [3]:
import math

def compute_ema_weight(current_time, event_time, window_size, alpha): 
    """ 
    Compute EMA weight for an event based on its age 
    Args: 
        current_time: Current timestamp in seconds 
        event_time: Event timestamp in seconds 
        window_size: Size of the decay window in seconds 
        alpha: Decay factor
     Returns: 
        Weight of the event at current_time 
    """ 
    time_diff = current_time - event_time 
    return math.pow(alpha, time_diff / window_size)

In [6]:
import time
compute_ema_weight(time.time(), time.time() - 3600, 86400, 0.5)

0.9715319411554642

In [15]:
import math
from typing import List, Tuple 
class AdaptiveEMA:
    def __init__(self, alpha: float, window_size : int):
        """
        Initialize EMA calculator with configurable parameters

        Args:
            alpha: Decay factor
            window_size: Decay window in seconds
        """
        self.alpha = alpha
        self.window_size = window_size

    def compute_ema(
        self,
        new_impressions : List[Tuple[int, float]],
        last_ema: float,
        last_update_time: int,
        current_time: int
    ) -> float:
        """
        Compute updated EMA incorporating new impressions and decayed historical value

        Args:
            new_impressions: List of (timestamp,value) tuples for new impressions
            last_ema: Previous EMA value
            last_update_time: Timestamp of last EMA update
            current_time: Current timestamp

        Returns:
            Updates EMA value
        """
        # Calculate EMA contributions from new impressions
        impression_sum = sum(
            value * math.pow(self.alpha, (current_time - timestamp) / self.window_size)
            for timestamp, value in new_impressions
        )

        # Decay the previous EMA value
        historical_decay = math.pow(
            self.alpha,
            (current_time - last_update_time) / self.window_size
        )
        decayed_previous_ema =  last_ema * historical_decay
        return impression_sum + decayed_previous_ema
        

In [17]:
# Example
ema_calculator = AdaptiveEMA(alpha=0.5, window_size=86400) #1-day window

# New impressions from last hour
import time
current_time = time.time()
recent_impressions = [
    (current_time - 3600, 1.0), # 1 hour ago
    (current_time - 1800, 1.0), # 30 minutes ago
    (current_time - 300, 1.0) # 5 minutes ago
]

updated_ema = ema_calculator.compute_ema(
    new_impressions = recent_impressions,
    last_ema = 2.5,
    last_update_time = current_time - 7200, # Last updated 2 hours ago
    current_time = current_time)
print(updated_ema)

5.314477054381648
